In [1]:
%pip install avro==1.10.0 kafka-python
%pip install kafka-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 4.7 MB/s eta 0:00:00
  Created wheel for avro: filename=avro-1.10.0-py3-none-any.whl size=96719 sha256=e224469c933c853f57c189f63dfae5c0cc8e5ce386a39b170c9ce96b0258a9e8
  Stored in directory: /root/.cache/pip/wheels/b5/45/02/59d22f799de5f011ddd515bf6e2e3b8c929ef56129008fe2b8
Successfully built avro


In [2]:
# import required libraries
from kafka import KafkaConsumer, KafkaProducer
import avro.schema
import avro.io
import io
import hashlib, json

In [3]:
def serialize(schema, obj):
    bytes_writer = io.BytesIO()
    encoder = avro.io.BinaryEncoder(bytes_writer)
    writer = avro.io.DatumWriter(schema)
    writer.write(obj, encoder)
    return bytes_writer.getvalue()

In [4]:
def deserialize(schema, raw_bytes):
    bytes_reader = io.BytesIO(raw_bytes)
    decoder = avro.io.BinaryDecoder(bytes_reader)
    reader = avro.io.DatumReader(schema)
    return reader.read(decoder)

In [5]:
schema_file = 'transaction.avsc'
txschema = avro.schema.parse(open(schema_file).read())
schema_file = 'submit.avsc'
submitschema = avro.schema.parse(open(schema_file).read())
schema_file = 'result.avsc'
resultschema = avro.schema.parse(open(schema_file).read())

In [8]:
# Connect to kafka broker running in your local host (docker). Change this to your kafka broker if needed
kafka_broker = 'lab.aimet.tech:9092'

In [9]:
producer = KafkaProducer(bootstrap_servers=[kafka_broker])

In [10]:
txconsumer = KafkaConsumer(
    'transaction',
     bootstrap_servers=[kafka_broker],
     enable_auto_commit=True,
     value_deserializer=lambda x: deserialize(txschema, x))
resultconsumer = KafkaConsumer(
    'result',
     bootstrap_servers=[kafka_broker],
     enable_auto_commit=True,
     value_deserializer=lambda x: deserialize(resultschema, x))

In [11]:
def gen_signature(txid, payer, payee, amount, token):
    o = {'txid': txid, 'payer': payer, 'payee': payee, 'amount': amount, 'token': token}
    return hashlib.md5(json.dumps(o, sort_keys=True).encode('utf-8')).hexdigest()

In [13]:
VID = 'V010948'
TOKEN = '2a7c55cfc9eb016fed2f7a38edc08d5e'

In [31]:
def findResult() :
    result = None
    for x in txconsumer:
        transaction = x.value
        signature = gen_signature(transaction['txid'], transaction['payer'], transaction['payee'], transaction['amount'], TOKEN)

        print("txid:", transaction['txid'], )
        print("payer:", transaction['payer'])
        print("payee:", transaction['payee'])
        print("amount:", transaction['amount'])
        print("signature:", signature)

        submit = {
            'vid': VID,
            'txid': transaction['txid'],
            'signature': signature
        }

        producer.send('submit', serialize(submitschema, submit))

        for x in resultconsumer:
            if transaction['txid'] == x.value['txid']:
                return x.value

In [32]:
print("result:", findResult())

txid: TX00818
payer: A97418
payee: A46843
amount: 303
signature: b492ed3936d146b6feb7b27f2e4111b2
result: {'timestamp': 1711440064, 'vid': 'V010948', 'txid': 'TX00818', 'checksum': '13b8eddd8e3caeb51612ce9bb0e6823a', 'code': 200, 'message': 'Confirm'}
